<a href="https://colab.research.google.com/github/sgu20191619/DACON/blob/main/%EC%B2%AD%EA%B2%BD%EC%B1%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

총추정광량(35) = (백색추정광량36 + 적색추정광량37 + 청색추정광량38)
-> LED랑 상관이 있음 

기준온도(32) = 난방온도(31) + 냉방온도(30) / 2 

난방부하 / 냉방부하는 모르겠음 

카메라상태 -> 필요없을듯 

화이트(21), 레드(23), 블루(26)의 동작상태는 201이면 켜짐 0이면 꺼짐인것 같음
각 작동남은시간은 초단위 
작동강도는 말그대로 강도  -> 강도가 0이면 무시해도 될듯? 

내부유동팬상태(16) 남은시간(17) 모두 같은 원리 
외부(18) (19) 도 마찬가지

난방상태(14) 작동남은시간(15)
냉방상태(12) 냉방작동(13)  -> 기준온도나 냉방온도 난방온도 있어서 의미없을듯?

일간누적분무량(11) -> 말 그대로 누적임 최근분무량에 영향받음(10) 
-> 2~1441까지 기록 -> 1442에서 0으로 초기화 

펌프상태(8) 남은시간(9) 역시 같은방식 201이 켜지는듯 



"펌프작동남은시간.1", "외부온도추정관측치" , "외부습도추정관측치", "펌프최근분무량", "펌프일간누적분무량"

In [ ]:
import random
import pandas as pd
import numpy as np
import os
import glob

import cv2
from time import time

import warnings
warnings.filterwarnings(action='ignore') 

import xgboost
from xgboost import plot_importance

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error,mean_absolute_error
from sklearn.linear_model import LinearRegression

from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import MinMaxScaler 
from sklearn.preprocessing import RobustScaler 
from sklearn.preprocessing import StandardScaler 
scaler_r = RobustScaler()
scaler_s = StandardScaler()
scaler_m = MinMaxScaler()
from sklearn.preprocessing import QuantileTransformer
quan = QuantileTransformer(output_distribution='normal')
#from sklearn.neighbors import _dist_metrics
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestClassifier

%matplotlib inline
import matplotlib.pyplot as plt
import statistics # 중앙값 

from collections import Counter 

#import autosklearn.regression
#from autosklearn.regression import AutoSklearnRegressor 

import seaborn as sns
from scipy import stats

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
all_input_list = sorted(glob.glob('/content/drive/MyDrive/open/train_input/*.csv'))
all_target_list = sorted(glob.glob('/content/drive/MyDrive/open/train_target/*.csv'))

In [ ]:
train_input_list = all_input_list
train_target_list = all_target_list

In [ ]:
test_input_list = sorted(glob.glob('/content/drive/MyDrive/open/test_input/*.csv'))
test_target_list = sorted(glob.glob('/content/drive/MyDrive/open/test_target/*.csv'))

In [ ]:
# 데이터 처리 

def IQR(df) :
  q3 = df.quantile(0.75)
  q1 = df.quantile(0.25)

  iqr = q3 - q1
  temp = df 
  temp = temp[((temp >  (q3 +1.5 *iqr)) | (temp < (q3 - 1.5 *iqr)))]
  temp = temp.where(temp == False, np.nan)
  #med = statistics.median(df)
  for i in temp.index :
    if i == 0 or i % 1440 == 0 :
      df[i] = df[i+1]
    else :
      df[i] = df[i-1]
  
  return df

first_df = pd.read_csv(train_input_list[0])
cols = first_df.columns 


In [ ]:
# 데이터 처리
'''
def IQR(df) :
  q3 = df.quantile(0.75)
  q1 = df.quantile(0.25)

  iqr = q3 - q1
  df = df.where(df < (q3 +1.5 *iqr), np.nan)
  df = df.where(df > (q1 -1.5 *iqr ),np.nan)

  #df = df.ewm(span= 10).mean()
  return df
  '''

'\ndef IQR(df) :\n  q3 = df.quantile(0.75)\n  q1 = df.quantile(0.25)\n\n  iqr = q3 - q1\n  df = df.where(df < (q3 +1.5 *iqr), np.nan)\n  df = df.where(df > (q1 -1.5 *iqr ),np.nan)\n\n  #df = df.ewm(span= 10).mean()\n  return df\n  '

In [ ]:
# 데이터 처리 
def IQR_2(df) :
  q3 = df.quantile(0.75)
  q1 = df.quantile(0.25)

  iqr = q3 - q1
  df = df.where(df < (q3 +1.5 *iqr), (q3 +1.5 *iqr))
  #df = df.where(df > (q1 -1.5 *iqr ),(q1 -1.5 *iqr))
  return df

In [ ]:
df = pd.DataFrame() # 데이터프레임
len_target = []

for i in train_target_list :
  if i == train_target_list[42] :
    continue
  target = pd.read_csv(i) # csv파일 읽고

  len_target.append(len(target)) # 케이스 별 타겟의 개수 
  df = pd.concat([df, target['rate']], ignore_index = False)

In [ ]:
first_df = pd.read_csv(train_input_list[0])
cols = first_df.columns 

In [ ]:
mean_data = pd.DataFrame()
feature_importance = cols

case = 0 
concate = []

for i in train_input_list :
  print(i)
  train = pd.read_csv(i)
  if i == train_input_list[42] :
    continue

  train.columns = cols
  train = pd.DataFrame(train)
  time = train[cols[0]]

  l = int(len(train)) // 1440
  j = 1 
  #day = pd.DataFrame()
  while j <= l :
    minute = train.iloc[1440*(j-1): 1440 * j] # 데이터프레임으로 하루 단위 추출 
    for num in feature_importance[1:]:
      #minute[num] = np.log1p(minute[num]) # 로그변환 
      if ('상태' not in num) and ('남은시간' not in num) and ('분무량' not in num):
        #print(num)
        minute[num] = IQR(minute[num])
    '''
    minute = minute.ewm(span=60).mean()
    #minute = minute.bfill()
    minute = pd.DataFrame(minute.mean())
    minute = minute.transpose()
    '''
    concate.append(minute)
    j += 1 
  
  temp = pd.concat(concate)
  temp = temp.ewm(span=240).mean()
  temp['시간'] = time  

  temp.dropna(subset=['시간'],inplace=True)
  temp['시간'] = pd.to_datetime(temp['시간'])
  temp['년월일'] = temp['시간'].dt.year.astype(str) + '-' + temp['시간'].dt.month.astype(str) + '-' + temp['시간'].dt.day.astype(str)
  temp['년월일'] = pd.to_datetime(temp['년월일'])
  temp_mean = temp.groupby('년월일').mean()
  temp_mean = temp_mean.reset_index(drop=True)
  #scaler = MinMaxScaler()
  #scaler.fit(temp_mean)

  #df_sam = scaler.transform(temp_mean)
  #df_sam = pd.DataFrame(df_sam, columns=temp_mean.columns)

  mean_data = pd.concat([mean_data, temp_mean])
                        
mean_data = pd.DataFrame(mean_data)

In [ ]:
mean_data.isna().sum()

In [ ]:
mean_data

,내부온도관측치,내부습도관측치,CO2관측치,EC관측치,외부온도관측치,외부습도관측치,펌프상태,펌프작동남은시간,최근분무량,일간누적분무량,...,카메라상태,냉방온도,난방온도,기준온도,난방부하,냉방부하,총추정광량,백색광추정광량,적색광추정광량,청색광추정광량
0,22.593001,27.069757,391.837875,0.914803,19.160300,10.080425,0.536039,0.357289,155.169885,415.542540,...,0.713405,22.375720,20.375720,21.375720,0.000000,1.450619,0.000000,0.000000,0.000000,0.000000
1,22.038630,33.017648,371.262451,0.908688,17.784645,10.915595,2.619885,1.565374,721.411530,5811.454110,...,0.818973,22.395832,20.395832,21.395832,0.000000,0.107268,0.000000,0.000000,0.000000,0.000000
2,21.905797,34.038837,356.329126,0.906506,18.065036,13.849074,2.481064,1.403217,710.777460,5921.586832,...,0.855688,22.395838,20.395838,21.395838,0.000000,0.335454,0.000000,0.000000,0.000000,0.000000
3,22.887668,27.416865,352.190017,0.909105,19.875512,20.367422,2.349204,1.420492,761.850665,6104.071177,...,0.698256,22.395837,20.395837,21.395837,0.000000,2.465703,0.000000,0.000000,0.000000,0.000000
4,23.551652,38.171576,379.072703,0.910420,21.957064,21.719805,2.348018,1.394359,718.633378,5898.784719,...,0.697917,22.395828,20.395828,21.395828,0.000000,5.069679,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20,24.670192,72.795813,498.760915,2.019799,24.471430,35.527233,3.654221,5.344731,498.836960,5222.672180,...,0.535514,21.813743,19.877919,20.562557,1.469862,25.111834,194.724997,141.178032,27.910641,24.442334
21,24.613385,73.594384,509.138141,1.953073,24.506287,37.902622,3.625680,3.732398,503.992034,4408.224674,...,0.521148,21.807984,19.872970,20.556333,1.594099,25.453215,194.760754,141.004857,27.555094,25.003255
22,24.478774,72.705324,495.606712,1.855380,24.285471,37.488086,3.902400,9.637901,4627.596502,8391.064739,...,0.580026,21.724568,19.761212,20.435020,1.548989,25.230885,189.038168,135.806006,27.437261,24.504704
23,24.135547,71.937475,490.258645,1.669997,24.110019,37.154066,3.758128,6.500356,1738.694750,11851.177506,...,0.570163,21.733352,19.773896,20.454855,1.911905,24.078635,192.247073,137.518559,28.399041,25.034965


In [ ]:
print(df)
mean_data['target'] = list(df[0])
mean_data

          0
0   0.50000
1   0.66667
2   0.60000
3  -0.12500
4   1.42857
..      ...
20 -0.07888
21 -0.15838
22 -0.47374
23 -0.51767
24 -0.50000

[1796 rows x 1 columns]


,내부온도관측치,내부습도관측치,CO2관측치,EC관측치,외부온도관측치,외부습도관측치,펌프상태,펌프작동남은시간,최근분무량,일간누적분무량,...,냉방온도,난방온도,기준온도,난방부하,냉방부하,총추정광량,백색광추정광량,적색광추정광량,청색광추정광량,target
0,22.593001,27.069757,391.837875,0.914803,19.160300,10.080425,0.536039,0.357289,155.169885,415.542540,...,22.375720,20.375720,21.375720,0.000000,1.450619,0.000000,0.000000,0.000000,0.000000,0.50000
1,22.038630,33.017648,371.262451,0.908688,17.784645,10.915595,2.619885,1.565374,721.411530,5811.454110,...,22.395832,20.395832,21.395832,0.000000,0.107268,0.000000,0.000000,0.000000,0.000000,0.66667
2,21.905797,34.038837,356.329126,0.906506,18.065036,13.849074,2.481064,1.403217,710.777460,5921.586832,...,22.395838,20.395838,21.395838,0.000000,0.335454,0.000000,0.000000,0.000000,0.000000,0.60000
3,22.887668,27.416865,352.190017,0.909105,19.875512,20.367422,2.349204,1.420492,761.850665,6104.071177,...,22.395837,20.395837,21.395837,0.000000,2.465703,0.000000,0.000000,0.000000,0.000000,-0.12500
4,23.551652,38.171576,379.072703,0.910420,21.957064,21.719805,2.348018,1.394359,718.633378,5898.784719,...,22.395828,20.395828,21.395828,0.000000,5.069679,0.000000,0.000000,0.000000,0.000000,1.42857
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20,24.670192,72.795813,498.760915,2.019799,24.471430,35.527233,3.654221,5.344731,498.836960,5222.672180,...,21.813743,19.877919,20.562557,1.469862,25.111834,194.724997,141.178032,27.910641,24.442334,-0.07888
21,24.613385,73.594384,509.138141,1.953073,24.506287,37.902622,3.625680,3.732398,503.992034,4408.224674,...,21.807984,19.872970,20.556333,1.594099,25.453215,194.760754,141.004857,27.555094,25.003255,-0.15838
22,24.478774,72.705324,495.606712,1.855380,24.285471,37.488086,3.902400,9.637901,4627.596502,8391.064739,...,21.724568,19.761212,20.435020,1.548989,25.230885,189.038168,135.806006,27.437261,24.504704,-0.47374
23,24.135547,71.937475,490.258645,1.669997,24.110019,37.154066,3.758128,6.500356,1738.694750,11851.177506,...,21.733352,19.773896,20.454855,1.911905,24.078635,192.247073,137.518559,28.399041,25.034965,-0.51767


In [ ]:
df_corr = mean_data.corr()['target']
df_corr = pd.DataFrame(df_corr.sort_values(ascending = False), columns = ['target'])
df_corr.style.background_gradient(cmap='viridis')

,target
target,1.000000
난방작동남은시간,0.068261
CO2관측치,0.059866
내부온도관측치,0.052043
난방상태,0.039819
기준온도,0.038059
백색광추정광량,0.022713
화이트 LED동작강도,0.007350
내부유동팬상태,0.006281
내부유동팬작동남은시간,0.003401


In [ ]:
df_corr[abs(df_corr['target'])>=0.24].index

Index(['target'], dtype='object')

In [ ]:
#feature = ['EC관측치', '백색광추정광량', '총추정광량', '화이트 LED동작강도', 'CO2관측치',
#       '외부환기팬작동남은시간', '난방부하', '외부온도관측치', '외부습도관측치', '내부습도관측치']
feature = ['난방작동남은시간', '내부온도관측치', '기준온도', '청색광추정광량', '외부온도관측치',
       '적색광추정광량', '카메라상태', '레드 LED동작강도', '난방부하', '외부환기팬작동남은시간', '내부습도관측치']
final = mean_data[feature]

In [ ]:
final

,난방작동남은시간,내부온도관측치,기준온도,청색광추정광량,외부온도관측치,적색광추정광량,카메라상태,레드 LED동작강도,난방부하,외부환기팬작동남은시간,내부습도관측치
0,0.000000,22.593001,21.375720,0.000000,19.160300,0.000000,0.713405,0.000000,0.000000,12.688858,27.069757
1,0.000000,22.038630,21.395832,0.000000,17.784645,0.000000,0.818973,50.374359,0.000000,22.919388,33.017648
2,0.000000,21.905797,21.395838,0.000000,18.065036,0.000000,0.855688,51.269317,0.000000,28.228922,34.038837
3,0.000000,22.887668,21.395837,0.000000,19.875512,0.000000,0.698256,51.298572,0.000000,44.683728,27.416865
4,0.000000,23.551652,21.395828,0.000000,21.957064,0.000000,0.697917,51.355692,0.000000,33.368980,38.171576
...,...,...,...,...,...,...,...,...,...,...,...
20,4.646009,24.670192,20.562557,24.442334,24.471430,27.910641,0.535514,20.392964,1.469862,16.450383,72.795813
21,7.516220,24.613385,20.556333,25.003255,24.506287,27.555094,0.521148,20.012518,1.594099,18.134568,73.594384
22,5.600728,24.478774,20.435020,24.504704,24.285471,27.437261,0.580026,20.193242,1.548989,20.505090,72.705324
23,5.153186,24.135547,20.454855,25.034965,24.110019,28.399041,0.570163,20.784117,1.911905,20.934525,71.937475


In [ ]:
params = {
    'objective' : 'reg:squarederror',
    'seed' : 2022
}

X = pd.DataFrame(final)
y = df
print(y)

'''
X_train, X_val, Y_train, Y_val = train_test_split(X,y, test_size= 0.1, random_state = 123) 
dtrain = xgboost.DMatrix(X_train, Y_train)
dval = xgboost.DMatrix(X_val,Y_val)

xgb_model = xgboost.train(params = params,num_boost_round = 2000,dtrain=dtrain, early_stopping_rounds = 100, evals = [ (dtrain,'train'),(dval,'eval')])

model.fit(X_train,Y_train)

preds = model.predict(X_val)
rmse = np.sqrt(mean_squared_error(Y_val, preds))
print("RMSE :%f" %(rmse) )
'''

          0
0   0.50000
1   0.66667
2   0.60000
3  -0.12500
4   1.42857
..      ...
20 -0.07888
21 -0.15838
22 -0.47374
23 -0.51767
24 -0.50000

[1796 rows x 1 columns]


'\nX_train, X_val, Y_train, Y_val = train_test_split(X,y, test_size= 0.1, random_state = 123) \ndtrain = xgboost.DMatrix(X_train, Y_train)\ndval = xgboost.DMatrix(X_val,Y_val)\n\nxgb_model = xgboost.train(params = params,num_boost_round = 2000,dtrain=dtrain, early_stopping_rounds = 100, evals = [ (dtrain,\'train\'),(dval,\'eval\')])\n\nmodel.fit(X_train,Y_train)\n\npreds = model.predict(X_val)\nrmse = np.sqrt(mean_squared_error(Y_val, preds))\nprint("RMSE :%f" %(rmse) )\n'

In [ ]:
from sklearn.model_selection import KFold

l_model = []
k_fold = KFold(n_splits = 5, shuffle=True, random_state = 123)
for train_idx, val_idx in k_fold.split(X) :
  model = xgboost.XGBRegressor(
     objective = 'reg:squarederror',
     max_depth = 6,
     n_estimators = 10000,
     eta = 0.008,
     learning_rate = 0.009, 
     num_boost_around = 5000,
     min_childe_weight = 1,
     seed = 2022,
     sub_sample = 0.9,
     colsample_bytree = 0.9,
     use_missing = False,
    ) 

  x_t = X.iloc[train_idx]
  y_t = y.iloc[train_idx]
  x_val = X.iloc[val_idx]
  y_val = y.iloc[val_idx]

  l_model.append(model.fit(x_t,y_t, eval_set= [(x_val,y_val)], early_stopping_rounds = 100))

[0]	validation_0-rmse:0.411948
Will train until validation_0-rmse hasn't improved in 100 rounds.
[1]	validation_0-rmse:0.409668
[2]	validation_0-rmse:0.409812
[3]	validation_0-rmse:0.407798
[4]	validation_0-rmse:0.40648
[5]	validation_0-rmse:0.405286
[6]	validation_0-rmse:0.40351
[7]	validation_0-rmse:0.402593
[8]	validation_0-rmse:0.400524
[9]	validation_0-rmse:0.398985
[10]	validation_0-rmse:0.397613
[11]	validation_0-rmse:0.396887
[12]	validation_0-rmse:0.39826
[13]	validation_0-rmse:0.397682
[14]	validation_0-rmse:0.396613
[15]	validation_0-rmse:0.396271
[16]	validation_0-rmse:0.397552
[17]	validation_0-rmse:0.39673
[18]	validation_0-rmse:0.39644
[19]	validation_0-rmse:0.395752
[20]	validation_0-rmse:0.395041
[21]	validation_0-rmse:0.395095
[22]	validation_0-rmse:0.395048
[23]	validation_0-rmse:0.396887
[24]	validation_0-rmse:0.397233
[25]	validation_0-rmse:0.396884
[26]	validation_0-rmse:0.396772
[27]	validation_0-rmse:0.396574
[28]	validation_0-rmse:0.396815
[29]	validation_0-rms

In [ ]:
for case in range(6) :
  preds = []
  test = pd.read_csv(test_input_list[case])
  test.columns = cols 
  time = test[cols[0]]
  l = int(len(test)) // 1440
  j = 1 
  day = []

  while j <= l :
    minute = test.iloc[1440*(j-1): 1440 * j] # 데이터프레임으로 하루 단위 추출 
    for num in feature_importance[1:]:
      #minute[num] = np.log1p(minute[num])
      if ('상태' or '남은시간' or '분무량') not in num:
        minute[num] = IQR(minute[num])
    day.append(minute)
    j += 1 
  
  temp = pd.concat(day)
  temp = temp.ewm(span=240).mean()
  temp['시간'] = time  

  temp.dropna(subset=['시간'],inplace=True)
  temp['시간'] = pd.to_datetime(temp['시간'])
  temp['년월일'] = temp['시간'].dt.year.astype(str) + '-' + temp['시간'].dt.month.astype(str) + '-' + temp['시간'].dt.day.astype(str)
  temp['년월일'] = pd.to_datetime(temp['년월일'])
  temp_mean = temp.groupby('년월일').mean()
  temp_mean = temp_mean.reset_index(drop=True)

  for model in l_model :
    test_df = temp_mean[feature]
    preds.append(model.predict(test_df))

  print(preds)
  y_test = pd.read_csv(test_target_list[case]) # 정답
  ans = pd.DataFrame(preds)
  ans = ans.mean()
  print(ans)
  submit_df = pd.read_csv(test_target_list[case])
  submit_df['rate'][0:] = ans[len(ans)-len(submit_df):]
  submit_df.to_csv(test_target_list[case],index = False)




[array([0.48506206, 0.5827528 , 0.539536  , 0.53594166, 0.5827528 ,
       0.54854935, 0.541152  , 0.541152  , 0.541152  , 0.541152  ,
       0.541152  , 0.541152  , 0.541152  , 0.541152  , 0.541152  ,
       0.541152  , 0.541152  , 0.541152  , 0.541152  , 0.541152  ,
       0.541152  , 0.541152  , 0.541152  , 0.5966629 , 0.541152  ,
       0.541152  , 0.541152  , 0.541152  , 0.541152  ], dtype=float32), array([0.34033394, 0.3388415 , 0.43470892, 0.437941  , 0.4438672 ,
       0.451307  , 0.34307897, 0.34307897, 0.34307897, 0.34307897,
       0.3450107 , 0.3492545 , 0.4260327 , 0.3492545 , 0.36299968,
       0.41979715, 0.36299968, 0.36299968, 0.41979715, 0.3492545 ,
       0.3492545 , 0.3492545 , 0.34340006, 0.4504569 , 0.34611124,
       0.34626186, 0.3492545 , 0.40605196, 0.3492545 ], dtype=float32), array([0.6475085 , 0.5352212 , 0.5352212 , 0.5342124 , 0.5352212 ,
       0.5052514 , 0.44029132, 0.44029132, 0.44029132, 0.44029132,
       0.44029132, 0.43585113, 0.43585113, 0.435851

In [ ]:
import zipfile
os.chdir("/content/drive/MyDrive/open/test_target/")
submission = zipfile.ZipFile("../submission.zip", 'w')
for path in test_target_list:
    path = path.split('/')[-1]
    submission.write(path)
submission.close()

In [ ]:
model2 = LinearRegression() # 모델 정의

# model.fit(X, y) 

# y_hat = model.predict(X) # y값 예측

# rmse = np.sqrt(mean_squared_error(y, y_hat))
# print("RMSE :%f" %(rmse) )

neg_mse = cross_val_score(model, X, y, scoring = "neg_mean_squared_error", cv = 5)
rmse_scores  = np.sqrt(-1 * neg_mse)
print(rmse_scores)
print(np.mean(rmse_scores))

[2.51119999 0.6489227  2.3708263  2.91494734 2.74834157]
2.238847580144367
